In [37]:
from sqlalchemy import create_engine, text
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk

In [17]:
# Database connection details
username = 'michals2'
password = 'Nv1VZRmuTxRvJn3u'
host = 'mysql.agh.edu.pl'
port = 3306
database = 'michals2'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

In [30]:
query = text("SELECT * FROM Perfumes") 

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

df = pd.DataFrame(perfumes)
df

,id,Name,Designer,URL,Gender,Accords,Description,ProsAndCons,TopNotes,MiddleNotes,BaseNotes,Longevity,Sillage
0,6001,Unicorn Milkshake,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,sweet (100%); vanilla (58.1656%); lactonic (55...,Unicorn Milkshake by The Dua Brand is a Floral...,,[],[],[],,
1,6002,Princeless Princess,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,fresh (100%); fresh spicy (82.6144%); sweet (8...,Princeless Princess by The Dua Brand is a Orie...,,[],[],[],,
2,6003,White Milk,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,vanilla (100%); musky (86.7568%); sweet (79.18...,White Milk by The Dua Brand is a fragrance for...,,[],[],[],,
3,6004,Popped Cherry,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,almond (100%); sweet (82.4996%); vanilla (79%)...,Popped Cherry by The Dua Brand is a Aromatic F...,,[],[],[],,
4,6005,Angelic Elixir,The Dua Brand,https://www.fragrantica.com/perfume/The-Dua-Br...,women and men,warm spicy (100%); woody (91.3583%); cinnamon ...,Angelic Elixir by The Dua Brand is a Aromatic ...,,[],[],[],,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,7110,Opus XII – Rose Incense,Amouage,https://www.fragrantica.com/perfume/Amouage/Op...,women and men,amber (100%); rose (89.033%); aromatic (82.659...,Opus XII – Rose Incense by Amouage is a Orient...,,"['Olibanum', 'Ink', 'elemi']","['Damask Rose', 'Olibanum']","['Myrrh', 'Sandalwood', 'Vanilla', 'Cedar']",,
1110,7111,Opus XV – King Blue,Amouage,https://www.fragrantica.com/perfume/Amouage/Op...,women and men,woody (100%); oud (88.5707%); amber (72.3008%)...,Opus XV – King Blue by Amouage is a Oriental W...,,"['Mandarin Orange', 'Black Currant', 'Pink Pep...","['Frankincense', 'Amber']","['Agarwood (Oud)', 'Leather', 'Oak', 'Patchoul...",,
1111,7112,Love Mimosa,Amouage,https://www.fragrantica.com/perfume/Amouage/Lo...,women,yellow floral (100%); aquatic (95.7145%); powd...,Love Mimosa by Amouage is a Oriental Floral fr...,"['Good for spring/summer', 'Bright and sunny',...","['Cascalone', 'Violet Leaf', 'Orris']","['Mimosa', 'Pear', 'Paradisone']","['Heliotrope', 'Ambroxan', 'Ylang-Ylang']",very weak: 32; weak: 41; moderate: 239; long l...,intimate: 95; moderate: 319; strong: 250; enor...
1112,7113,Myths Man,Amouage,https://www.fragrantica.com/perfume/Amouage/My...,men,smoky (100%); aromatic (85.3538%); woody (80.5...,Myths Man by Amouage is a Oriental fragrance f...,"['Masterpiece and work of art', 'Unique and pl...","['Chrysanthemum', 'Orris']","['elemi', 'Rum', 'Vetiver', 'Rose']","['Ash', 'Leather', 'Labdanum']",very weak: 38; weak: 39; moderate: 111; long l...,intimate: 57; moderate: 311; strong: 661; enor...


In [ ]:
query = text("SELECT Accords FROM Perfumes") 

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

df = pd.DataFrame(perfumes)

accord_counter = Counter()

for row in df['Accords'].dropna():
    matches = re.findall(r'([\w\s\-]+)\s*\(\d', row)
    for accord in matches:
        accord_counter[accord.strip()] += 1

accord_freq_df = pd.DataFrame(accord_counter.items(), columns=["Accord", "Count"]).sort_values(by="Count", ascending=False)

accord_freq_df.to_excel("accord_frequencies.xlsx", index=False)


In [ ]:
query = text("SELECT Description FROM Perfumes") 

nltk.download('stopwords')

stop_words = set(stopwords.words("english"))

with engine.connect() as connection:
    result = connection.execute(query)
    perfumes = [dict(row._mapping) for row in result]

word_counter = Counter()

for desc in df['Description'].dropna():
    words = re.findall(r'\b\w+\b', desc.lower())
    filtered_words = [word for word in words if word not in stop_words]
    word_counter.update(filtered_words)

word_freq_df = pd.DataFrame(word_counter.items(), columns=["Word", "Count"]).sort_values(by="Count", ascending=False)

word_freq_df.to_excel("description_word_counts_filtered.xlsx", index=False)

print(word_freq_df.head(10))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jakubwasik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


           Word  Count
41        notes   2506
7     fragrance   2177
86      perfume   1028
11     launched    901
95    português    859
89      español    859
102  українська    859
101     العربية    859
100      română    859
96     ελληνικά    859
